In [18]:
from bs4 import BeautifulSoup
import requests

In [19]:
# read the href_set.txt file and save the hrefs in a list
urls = []
with open('href_set.txt', 'r') as f:
    for line in f:
        urls.append(line.strip())

In [22]:
def scrape_ingredients(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the title of the recipe
    title = soup.find('title').get_text().strip()

    # Try to find the ingredients section by looking for "Ingredients" header
    ingredients = []
    
    # Look for the header that contains "Ingredients"
    ingredients_section = soup.find(lambda tag: tag.name.startswith('h') and 'ingredient' in tag.get_text().lower())
    
    if ingredients_section:
        # Try finding an unordered list (<ul>) or ordered list (<ol>) immediately following the header
        ingredients_list = ingredients_section.find_next('ul') or ingredients_section.find_next('ol')

        if ingredients_list:
            # Extract the ingredients from the list items
            ingredients = [li.get_text().strip() for li in ingredients_list.find_all('li')]
        else:
            # Backup: try to find ingredients in nearby <p> tags
            ingredients_paragraphs = ingredients_section.find_next_siblings('p', limit=10)
            for paragraph in ingredients_paragraphs:
                ingredients.append(paragraph.get_text().strip())
    
    return title, ingredients

In [23]:
with open('cookie_ingredients.txt', 'w') as file:
    for url in urls:
        try:
            name, ingredients = scrape_ingredients(url)
            if ingredients:
                file.write(f'{name}, {", ".join(ingredients)}\n')
            else:
                file.write(f'{name}, No ingredients found\n')
        except Exception as e:
            print(f"Failed to scrape {url}: {e}")